# Upwind scheme for the Wave System

## The Wave System on the square

We consider the following Wave system with periodic boundary conditions

$$
\left\{\begin{array}{l}
\partial_t p + c^2\nabla\cdot\vec q = 0\\
\partial_t \vec q + \quad\vec\nabla p = 0
\end{array}\right..
$$

The wave system can be written in matrix form 
$$
\partial_t
\left(
\begin{array}{c}
 p \\
 \vec q
\end{array}\right)
+
\left(\begin{array}{cc}
 0   & c^2 \nabla \cdot \\ 
 \vec\nabla & 0
\end{array}\right)
\left(\begin{array}{c}
 p \\ 
 \vec q
\end{array}\right)=0
$$

In $d$ space dimensions the wave system is an hyperbolic system of $d+1$ equations
$$
\partial_t U +\sum_{i=1}^d A_i\partial_{x_i} U=0,\quad U={}^t(p,\vec q)
$$
where the jacobian matrix is
$$
A(\vec n)=\sum_{i=1}^d n_i A_i =
\left(
\begin{array}{cc}
 0              & c^2 {}^t\vec n \\ 
 \vec n & 0
\end{array}\right),\quad \vec n\in\mathbb{R}^d.
$$
has $d+1$ eigenvalues $-c,0,\dots,0,c$.

The wave system also takes the conservative form
$$
\partial_t U + \nabla\cdot F(U)=0,
$$
where the flux matrix $F$ is defined by
$$
F(U)\vec n=A(\vec n)U,\quad \vec n\in\mathbb{R}.
$$

On the square domain $\Omega= [0,1]\times [0,1]$ we consider the initial data $(p_0,\vec q_0)$ 

$$
\left\{\begin{array}{l}
p_0(x,y)=constant\\
q_{x0}(x,y)= \sin(\pi x) \cos(\pi y)\\
q_{y0}(x,y)=-\sin(\pi y) \cos(\pi x)
\end{array}\right..
$$  

The initial data $(p_0,q_x,q_y)$ is a stationary solution of the wave system.

$$
||\vec q_0||^2=\sin(\pi x)^2 \cos(\pi y)^2+\sin(\pi y)^2 \cos(\pi x)^2=\frac{1}{2}\sin(\pi (x + y))^2+\frac{1}{2}\sin(\pi (x-y))^2
$$


## The Upwind scheme for the Wave System

The domain $\Omega$ is decomposed into cells $C_i$.

- $|C_i|$ is the measure of the cell $C_i$.

- $f_{ij}$ is the interface between two cells $C_i$ and $C_j$. 

- $s_{ij}$ is the measure of the interface $f_{ij}$.

- $d_{ij}$ is the distance between the centers of mass of the two cells $C_i$ and $C_j$.

The semi-discrete colocated finite volume equation is
$$
\partial_t U + \frac{1}{|C_i|} \sum s_{ij}F_{ij}=0,
$$
where
$U_i$ is the approximation of $U$ in the cell $C_i$.

$F_{ij}$ is a numerical approximation of the outward normal interfacial flux from cell $i$ to cell $j$ usually in the upwind form
$$
F_{ij}=\frac{F(U_i)+F(U_j)}{2}\vec n-D(\vec n)\frac{U_i-U_j}{2}.
$$
In the case of the upwind scheme the upwind matrix is
$$
D_{upwind}(\vec n)=|A(\vec n)| = 
c
\left(
\begin{array}{cc}
1 & 0 \\
0 & \vec n\otimes\vec n
\end{array}\right).
$$

## Some bibliographical remarks

Most of the literature on finite volume methods for hyperbolic equations focus on scalar equation whilst we are interested in systems of equation. However the following references do apply to the wave system and the upwind scheme.

- The upwind scheme converges on general meshes with order 1/2  
  *J.-P. Vila, P. Villedieu, Convergence de la méthode des volumes finis pour les systèmes de Friedrichs, C. R. Acad. Sci. Paris, t. 325, Série I, 1997.*

Stationary regimes are of great importance in several application. In particular in the wave system the stationary momentum should satisfy $\partial_x q_x+\partial_y q_y=0$. However at the discrete level, the stationary regime can be very different.

- Study of the kernel of the discrete divergence, which corresponds to the discrete stationary regimes of the upwind scheme  
  *S. Dellacherie, P. Omnes, F. Rieper, The influence of cell geometry on the Godunov scheme applied to the linear wave equation, J. Comp. Phys., 229(14), pp. 5315-5338, 2010*
    - On rectangular meshes the discrete kernel is $\partial_x q_x=\partial_y q_y=0$ which is a much smaller space than $\partial_x q_x+\partial_y q_y=0$.
    - On triangular meshes the discrete kernel is much larger. In practical simulation we observe convergence towards the correct stationary regime.

- Relationships between stationarity preservation, low Mach precision and vorticity preservation  
  *Wasilij Barsukow: Stationarity preserving schemes for multi-dimensional linear systems, 2017, accepted in Math. Comp*

## The script

```python
#Condition initiale
pressure_field, velocity_field = initial_conditions_wave_system(my_mesh)

#Pas de temps
dt = cfl * dx_min / c0

#Matrice des systèmes linéaires
divMat=computeDivergenceMatrix(my_mesh,nbVoisinsMax,dt,test_bc)

# Construction du vecteur inconnu
Un=cdmath.Vector(nbCells*(dim+1))
for k in range(nbCells):
    Un[k*(dim+1)+0] =     pressure_field[k]
    Un[k*(dim+1)+1] =rho0*velocity_field[k,0]
    Un[k*(dim+1)+2] =rho0*velocity_field[k,1]


# Création du solveur linéaire
LS=cdmath.LinearSolver(divMat,Un,iterGMRESMax, precision, "GMRES","ILU")

# Time loop
while (it<ntmax and time <= tmax and not isStationary):
    LS.setSndMember(Un)
    Un=LS.solve();
    Un.writeVTK
    
# Automatic postprocessing :  save 2D picture and plot diagonal data
#===========================
diag_data=VTK_routines.Extract_field_data_over_line_to_numpyArray(my_ResultField,[0,1,0],[1,0,0], resolution)
plt.legend()
plt.xlabel('Position on diagonal line')
plt.ylabel('Value on diagonal line')
if len(sys.argv) >1 :
    plt.title('Plot over diagonal line for finite volumes \n for Wave system on a 2D square '+my_mesh.getName())
    plt.plot(curv_abs, diag_data, label= str(nbCells)+ ' cells mesh')
    plt.savefig("FiniteVolumes2D_square_ResultField_"+str(nbCells)+ '_cells'+"_PlotOverDiagonalLine.png")

```

## Numerical results for upwind scheme on cartesian meshes

### Cartesian meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DWaveSystemUpwind/squareWithSquares_2.png) | ![](2DWaveSystemUpwind/squareWithSquares_3.png)  | ![](2DWaveSystemUpwind/squareWithSquares_4.png) 


### Velocity initial data (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwind15x15_velocity_initial.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwind31x31_velocity_initial.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwind51x51_velocity_initial.png) 


### Stationary velocity (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwind15x15_velocity_Stat.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwind31x31_velocity_Stat.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwind51x51_velocity_Stat.png) 


### Stationary velocity (x component)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwind15x15_velocityX_Stat.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwind31x31_velocityX_Stat.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwind51x51_velocityX_Stat.png) 


### Stationary velocity (y component)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwind15x15_velocityY_Stat.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwind31x31_velocityY_Stat.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwind51x51_velocityY_Stat.png) 


### Convergence on velocity

![](2DWaveSystemUpwind/meshSquareWithSquares_Velocity_2DWaveSystemUpwind_ConvergenceCurve.png)

## Numerical results for upwind scheme on triangular meshes

### Triangular meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DWaveSystemUpwind/squareWithTriangles_2.png) | ![](2DWaveSystemUpwind/squareWithTriangles_3.png)  | ![](2DWaveSystemUpwind/squareWithTriangles_4.png) 


### Velocity initial data (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithTriangles224_velocity_initial.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithTriangles934_velocity_initial.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithTriangles6422_velocity_initial.png) 


### Stationary velocity  (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithTriangles224_velocity_Stat.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithTriangles934_velocity_Stat.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithTriangles6422_velocity_Stat.png) 


### Convergence on stationary velocity

![](2DWaveSystemUpwind/squareWithTriangles_Velocity_2DWaveSystemUpwind_Triangles_ConvergenceCurve.png)

## Numerical results for upwind scheme on checkerboard meshes

### Checkerboard meshes

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DWaveSystemUpwind/checkerboard_17x17.png) | ![](2DWaveSystemUpwind/checkerboard_33x33.png)  | ![](2DWaveSystemUpwind/checkerboard_65x65.png) 


### Velocity initial data (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithCheckerboard204_velocity_initial.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithCheckerboard724_velocity_initial.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithCheckerboard2724_velocity_initial.png) 


### Stationary velocity (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithCheckerboard204_velocity_Stat.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithCheckerboard724_velocity_Stat.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithCheckerboard2724_velocity_Stat.png) 


### Convergence on stationary velocity

![](2DWaveSystemUpwind/squareWithCheckerboard_Velocity_2DWaveSystemUpwind_Checkerboard_ConvergenceCurve.png)

## Numerical results for upwind scheme on flat triangle meshes

### Flat cross triangle meshes (from a $(n,n^2)$ rectangular grid)

mesh 1 | mesh 2 | mesh 3
     - | -    - | -
![](2DWaveSystemUpwind/squareWithCrossTriangles_00.png) | ![](2DWaveSystemUpwind/squareWithFlatTriangles_0.png)  | ![](2DWaveSystemUpwind/squareWithFlatTriangles_1.png) 


### Velocity initial data (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithFlatCrossTriangles500_velocity_initial.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithFlatCrossTriangles5324_velocity_initial.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithFlatCrossTriangles37044_velocity_initial.png) 


### Stationary velocity (magnitude)

result 1 | result 2 | result 3
       - | -      - | -
![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithFlatCrossTriangles500_velocity_Stat.png) | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithFlatCrossTriangles5324_velocity_Stat.png)  | ![](2DWaveSystemUpwind/WaveSystem2DUpwindsquareWithFlatCrossTriangles37044_velocity_Stat.png) 


### Convergence on stationary velocity

![](2DWaveSystemUpwind/squareWithFlatCrossTriangles_Velocity_2DWaveSystemUpwind_ConvergenceCurve.png)